# Hybrid Embeddings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import load_dataset
from qdrant_client import models, QdrantClient
from fastembed import SparseTextEmbedding, TextEmbedding

from src import load_data, embeddings


c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Params

In [3]:
INDEX = "ragquas-hybrid-01"
RETRIEVE_K=5
SMPL_SIZE = 25  # int or None

# Environment variables

In [4]:
load_dotenv()


True

# Clients

In [5]:
client_qdrant = QdrantClient(":memory:") 


# Load and Prepare Data

In [6]:
df_ragquas = load_data.ragquas()
df_ragquas.head(2)

,topic,answer,question,variant,context_1,context_2,context_3,context_4,context_5,link_1,link_2,link_3,link_4,link_5,text_1,text_2,text_3,text_4,text_5
0,reclamaciones,La opción más fácil y eficaz para reclamar una...,¿Cuál es la forma más fácil de reclamar cuando...,question_1,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,
1,reclamaciones,La opción más fácil y eficaz para reclamar una...,hola mi vuelo a Alemania se ha retrasado mucho...,question_2,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,


In [7]:
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [8]:
if SMPL_SIZE:
    df_docs_smpl = df_docs.iloc[:SMPL_SIZE]
else:
    df_docs_smpl = df_docs

## Hybrid Search in Qdrant

[FastEmbed Supported models](sentence-transformers/paraphrase-multilingual-mpnet-base-v2)

In [9]:

documents = [
    "You should stay, study and sprint.",
    "History can only prepare us to be surprised yet again.",
]

encoder_sp = SparseTextEmbedding(model_name="Qdrant/bm25")
sp_embeddings = list(encoder_sp.embed(documents))


print(f"{sp_embeddings[0].values=}")
print(f"{sp_embeddings[0].indices=}")

encoder_dn = TextEmbedding(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
dn_embeddings = list(encoder_dn.embed(documents))

print(f"{len(dn_embeddings)=}")
print(f"{len(dn_embeddings[0])=}")

sp_embeddings[0].values=array([1.67868852, 1.67868852, 1.67868852])
sp_embeddings[0].indices=array([1881538586,  150760872, 1932363795])


C:\Users\manuelalberto.romero\AppData\Local\Temp\ipykernel_49048\3171690876.py:13: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  encoder_dn = TextEmbedding(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\manuelalberto.ro

len(dn_embeddings)=2
len(dn_embeddings[0])=768


# Encode

In [10]:
lst_embeddings_hyb = []

corpus_encoded_sp = list(encoder_sp.embed(df_docs_smpl['text_']))
corpus_encoded_dn = list(encoder_dn.embed(df_docs_smpl['text_']))

for idx in range(len(corpus_encoded_sp)):
    sp_vec = corpus_encoded_sp[idx]
    dn_vec = corpus_encoded_dn[idx]

    lst_embeddings_hyb.append({
        "sparse": models.SparseVector(
            indices=sp_vec.indices,
            values=sp_vec.values
            ),
        "dense": dn_vec.tolist()
        }
    )

lst_pts_hyb = embeddings.create_index_points(
    lst_embeddings_hyb,
    df_docs_smpl,
)

lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts_hyb)

print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=25


# Index

In [11]:
client_qdrant.create_collection(
    collection_name=INDEX,
    vectors_config={
        "dense": models.VectorParams(
            size=encoder_dn.embedding_size, 
            distance=models.Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(),
        }
)

client_qdrant.upsert(
    collection_name=INDEX,
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Query

In [ ]:
# Dense only
_q_text = df_docs.iloc[0:1]['question']

_q_vec_dn = next(encoder_dn.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        INDEX,
        _q_vec_dn,
        retrieve_k=1,
        using='dense'  # Name in StructPoint and Collection
)



Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='a8cbb9e2-551c-49c6-aba4-b5bb84f3777b'
point.score=0.49639524378939925
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelación, porque se desconocen los derechos como pasajeros y la manera de realizar el trámite. 
    Por ello, aquí van 5 reclamadores online que te ayudarán a conseguir una indemnización por tu vuelo, sin que tengas que desplazarte ni hacer papeleos.
    Descubre más historias en Business Insider España.

Volar es si...
idx: 0, j: 1
------------------------------


[Hybrid Queries](https://qdrant.tech/documentation/concepts/hybrid-queries/)

Prefecth is a component that allows to run multiple queries in parallel and then combine the results using a fusion algorithm.

Reciprocal Rank Fusion
![RRF](https://qdrant.tech/docs/fusion-idea.png)

In [ ]:
_q_text = df_docs_smpl.iloc[0:1]['question']

# Get Dense and sparse Vector
_q_vec_dn = next(encoder_dn.embed(_q_text))
_q_encoded_sp = next(encoder_sp.embed(_q_text))
_q_vec_sp = models.SparseVector(
    indices=_q_encoded_sp.indices,
    values=_q_encoded_sp.values
    )

prefetch = [
        models.Prefetch(
            query=_q_vec_dn,
            using="dense",
            limit=3,
        ),
        models.Prefetch(
            query=_q_vec_sp,
            using="sparse",
            limit=3,
        ),
    ]
_q_rrf = models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
results = client_qdrant.query_points(
        INDEX,
        prefetch=prefetch,
        query=_q_rrf,
        with_payload=True,
        limit=3,
    )

for point in results.points:
    doc_retrieved = point.payload['text']
    print(f"{point.id=}")
    print(f"{point.score=}")
    print(f"Doc: {doc_retrieved[:500]}...")
    print(f"idx: {point.payload['index']}, j: {point.payload['text_j']}")
    print("-"*30)


point.id='a8cbb9e2-551c-49c6-aba4-b5bb84f3777b'
point.score=0.8333333333333333
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelación, porque se desconocen los derechos como pasajeros y la manera de realizar el trámite. 
    Por ello, aquí van 5 reclamadores online que te ayudarán a conseguir una indemnización por tu vuelo, sin que tengas que desplazarte ni hacer papeleos.
    Descubre más historias en Business Insider España.

Volar es si...
idx: 0, j: 1
------------------------------
point.id='d0268e72-b7b6-4e92-84f8-5b42b9c2f77a'
point.score=0.75
Doc: Conoce las compensaciones cuando se retrasa tu vuelo

Llegas al aeropuerto y tu vuelo va con retraso, ¿conoces cuáles son tus derechos y si puedes pedir reclamar una compensación? Desde AirHelp te ayudamos con los trámites para que tu viaje no se convierta en una pesadilla y puedas conocer todos l